In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.pylab as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import PIL
from PIL import ImageFilter
import cv2
import itertools
import random
import keras
import imutils
from imutils import paths
import os
from keras import optimizers
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import callbacks
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose
from keras import backend as K

%matplotlib inline

In [ ]:
def pil_image(img_path):
    pil_im =PIL.Image.open(img_path).convert('L')
    pil_im=pil_im.resize((105,105))
    return pil_im

### Adding Noise to image

In [ ]:
def noise_image(pil_im):
    img_array = np.asarray(pil_im)
    mean = 0.0   # some constant
    std = 5   # some constant (standard deviation)
    noisy_img = img_array + np.random.normal(mean, std, img_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255)
    noise_img = PIL.Image.fromarray(np.uint8(noisy_img_clipped)) # output
    noise_img=noise_img.resize((105,105))
    return noise_img

### Adding Blur to image 

In [ ]:
def blur_image(pil_im):
    blur_img = pil_im.filter(ImageFilter.GaussianBlur(radius=3)) # ouput
    #imshow(blur_img)
    blur_img=blur_img.resize((105,105))
    return blur_img

### Rotation

In [ ]:
def affine_rotation(img):
    
    #img=cv2.imread(img_path,0)
    rows, columns = img.shape

    point1 = np.float32([[10, 10], [30, 10], [10, 30]])
    point2 = np.float32([[20, 15], [40, 10], [20, 40]])

    A = cv2.getAffineTransform(point1, point2)

    output = cv2.warpAffine(img, A, (columns, rows))
    affine_img = PIL.Image.fromarray(np.uint8(output)) # affine rotated output
    #imshow(output)
    affine_img=affine_img.resize((105,105))
    return affine_img

### Gradient

In [ ]:
def gradient_fill(image):
    laplacian = cv2.Laplacian(image,cv2.CV_64F)
    laplacian = cv2.resize(laplacian, (105, 105))
    return laplacian

### DataSet

In [ ]:
data_path = "Dataset/"
data=[]
labels=[]
imagePaths = sorted(list(paths.list_images(data_path)))
random.seed(42)
random.shuffle(imagePaths)

https://fonts.google.com/specimen/Oswald
https://fonts.google.com/specimen/Roboto
https://fonts.google.com/specimen/Open+Sans
https://fonts.google.com/specimen/Ubuntu
https://fonts.google.com/specimen/PT+Serif
https://fonts.google.com/specimen/Dancing+Script
https://fonts.google.com/specimen/Fredoka+One
https://fonts.google.com/specimen/Arimo
https://fonts.google.com/specimen/Noto+Sans
https://fonts.google.com/specimen/Patua+One

In [ ]:
def conv_label(label):
    label_dict = {
        'Oswald': 0,
        'Roboto': 1,
        'Open+Sans': 2,
        'Ubuntu': 3,
        'PT+Serif': 4,
        'Dancing+Script': 5,
        'Fredoka+One': 6,
        'Arimo': 7,
        'Noto+Sans': 8,
        'Patua+One': 9
    }
    return label_dict.get(label, -1)  # Returns -1 if label is not found


In [ ]:
augument=["blur","noise","affine","gradient"]
a=itertools.combinations(augument, 4)

for i in list(a): 
    print(list(i))

In [ ]:
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    label = conv_label(label)
    pil_img = pil_image(imagePath)  
    
    
    org_img = img_to_array(pil_img)  
    data.append(org_img)
    labels.append(label)
    
    augment_functions = {
        'noise': noise_image,  
        'blur': blur_image,    
        'affine': lambda img: affine_rotation(np.array(img)),  
        'gradient': lambda img: gradient_fill(np.array(img)),  
    }
    
    
    augmentations = ["noise", "blur", "affine", "gradient"]
    
    for l in range(1, len(augmentations) + 1):
        for combination in itertools.combinations(augmentations, l):
            temp_img = pil_img
            for aug in combination:
                temp_img = augment_functions[aug](temp_img)
            temp_img = img_to_array(temp_img)
            data.append(temp_img)
            labels.append(label)


In [ ]:
data = np.asarray(data, dtype="float") / 255.0
labels = np.array(labels)
print("Success")

(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [ ]:
trainY = to_categorical(trainY, num_classes=5)
testY = to_categorical(testY, num_classes=5)

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True)
K.set_image_dim_ordering('tf')


In [ ]:
def create_model():
  model=Sequential()

  # Cu Layers 
  model.add(Conv2D(64, kernel_size=(48, 48), activation='relu', input_shape=(105,105,1)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, kernel_size=(24, 24), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2DTranspose(128, (24,24), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  model.add(Conv2DTranspose(64, (12,12), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  #Cs Layers
  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Flatten())

  model.add(Dense(4096, activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(4096,activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(2383,activation='relu'))

  model.add(Dense(5, activation='softmax'))
 
  return model

In [ ]:
batch_size = 128
epochs = 50
model= create_model()
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [ ]:
early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')

filepath="top_model.h5"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [early_stopping,checkpoint]

In [ ]:
model.fit(trainX, trainY,shuffle=True,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(testX, testY),callbacks=callbacks_list)

In [ ]:
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from keras.models import load_model
model = load_model('top_model.h5')

In [ ]:
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Now we take a sample image and try to do the same and display the results!



In [ ]:

from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from matplotlib import cm



def rev_conv_label(label):
    labels_dict = {
        0: 'Oswald',
        1: 'Roboto',
        2: 'Open+Sans',
        3: 'Ubuntu',
        4: 'PT+Serif',
        5: 'Dancing+Script',
        6: 'Fredoka+One',
        7: 'Arimo',
        8: 'Noto+Sans',
        9: 'Patua+One'
    }
    return labels_dict.get(label, "Unknown Label")

# Load and preprocess the image
img_path = "sample/sample.jpg"
pil_im = PIL.Image.open(img_path).convert('L')  
pil_im = blur_image(pil_im)  
org_img = img_to_array(pil_im)  
org_img = np.expand_dims(org_img, axis=0)  
org_img = org_img.astype("float32") / 255.0  

# Predict the label of the image
y_pred = model.predict(org_img)
y_pred_label = np.argmax(y_pred, axis=1)[0]  

# Convert numeric label to actual font name
label = rev_conv_label(y_pred_label)

# Visualize the result
fig, ax = plt.subplots(1)
ax.imshow(pil_im, interpolation='nearest', cmap=cm.gray)
ax.text(5, 5, label, bbox={'facecolor': 'white', 'pad': 10})
plt.show()
